In [1]:
import csv
import os
import cv2
import sklearn
import numpy as np

## 第一步：读取driving_log.csv文件，然后读取每一行

In [2]:
samples = []

with open(r'./data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        samples.append(line)

In [3]:
from sklearn.model_selection import train_test_split
train_samples, validation_samples = train_test_split(samples, test_size=0.2)

In [4]:
len(train_samples),len(validation_samples)

(4060, 1016)

In [5]:
train_samples[0]

['E:\\2.Special Personal\\Self-Driving Car\\P3_Behavioral Cloning\\Under-Way\\CarND-Behavioral-Cloning-P3-master\\data\\IMG\\center_2017_12_26_11_26_10_919.jpg',
 'E:\\2.Special Personal\\Self-Driving Car\\P3_Behavioral Cloning\\Under-Way\\CarND-Behavioral-Cloning-P3-master\\data\\IMG\\left_2017_12_26_11_26_10_919.jpg',
 'E:\\2.Special Personal\\Self-Driving Car\\P3_Behavioral Cloning\\Under-Way\\CarND-Behavioral-Cloning-P3-master\\data\\IMG\\right_2017_12_26_11_26_10_919.jpg',
 '0',
 '1',
 '0',
 '30.19031']

## 第二步：获取每张图片的路径、方向盘，作为训练集和标签。

In [6]:
def generator(samples, batch_size=32):
    
    num_samples = len(samples)
    images_and_angles = [
        (0, 0.0),    # central camera
        (1, 0.2),   # left camera
        (2, -0.2),  # right camera
        ]
    
    while 1: # Loop forever so the generator never terminates
        sklearn.utils.shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            angles = []
            for batch_sample in batch_samples:
                for batch_sample_index, angle_adjust in images_and_angles:
                    name = './data/IMG/' + batch_sample[batch_sample_index].split('\\')[-1]
                    center_image = cv2.imread(name)
                    center_angle = float(batch_sample[3]) + angle_adjust
                    images.append(center_image)
                    angles.append(center_angle)

            augmented_images, augmented_angles = [], []
            for image, angle in zip(images, angles):
                augmented_images.append(image)
                augmented_angles.append(angle)
                augmented_images.append(cv2.flip(image, 1))
                augmented_angles.append(angle*-1.0)

            X_train = np.array(augmented_images)
            y_train = np.array(augmented_angles)
            yield sklearn.utils.shuffle(X_train, y_train)

In [7]:
# compile and train the model using the generator function
train_generator = generator(train_samples, batch_size=32)
validation_generator = generator(validation_samples, batch_size=32)

In [8]:
validation_generator

<generator object generator at 0x00000000084488E0>

In [ ]:
next(validation_generator)

## 第三步：建立模型

In [10]:
from keras.models import Sequential
from keras.layers import Convolution2D, Flatten, Dense, Activation, Lambda, Cropping2D
from keras.layers.pooling import MaxPooling2D

Using TensorFlow backend.


In [ ]:
model = Sequential()
model.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((70,25),(0,0))))
model.add(Convolution2D(24,(5,5), strides=(2,2), activation='relu'))
model.add(Convolution2D(36,(5,5), strides=(2,2), activation='relu'))
model.add(Convolution2D(48,(5,5), strides=(2,2), activation='relu'))
model.add(Convolution2D(64,(3,3), activation='relu'))
model.add(Convolution2D(64,(3,3), activation='relu'))
model.add(Flatten())
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(1))


model.compile(loss='mse', optimizer='adam')
model.fit_generator(
        generator=train_generator,
        samples_per_epoch=len(train_samples),   # 3 images per sample
        validation_data=validation_generator,
        nb_val_samples=len(validation_samples),
        nb_epoch=3)

model.save('model.h5')

D:\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:21: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
D:\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:21: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(generator=<generator..., validation_data=<generator..., steps_per_epoch=4060, epochs=3, validation_steps=1016)`


Epoch 1/3
4060/4060 [==============================] - 2233s - loss: 0.0087 - val_loss: 0.0193
Epoch 2/3
4060/4060 [==============================] - 2231s - loss: 0.0027 - val_loss: 0.0189
Epoch 3/3
4059/4060 [============================>.] - ETA: 0s - loss: 0.0018

In [12]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, 160, 320, 3)       0         
_________________________________________________________________
cropping2d_1 (Cropping2D)    (None, 65, 320, 3)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 31, 158, 24)       1824      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 77, 36)        21636     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 5, 37, 48)         43248     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 3, 35, 64)         27712     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 1, 33, 64)         36928     
__________